In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import sqlite3
import mysql.connector
import math

In [2]:
conn = mysql.connector.connect(user='root', password='password', host='127.0.0.1', port='3306',database='expression_atlas2', auth_plugin='mysql_native_password')
mycursor = conn.cursor()

# check the connection
if conn.is_connected():
    print("connection succesfull")
else:
    print("no connection")


connection succesfull


In [ ]:
assaysql = "SELECT assay_id, peptide_id, quantification FROM peptide_to_assay"
assayData = pd.read_sql_query(assaysql, conn)
assayData.head()

In [ ]:
assaytissuesql = "SELECT assay_id, tissue_id FROM tissue_to_assay"
assaytissueData = pd.read_sql_query(assaytissuesql, conn)
assaytissueData.head()

In [ ]:
tissuesql = "SELECT tissue_id, tissue_name,cell_type, disease_status FROM tissue"
tissueData = pd.read_sql_query(tissuesql, conn)
tissueData.head(15)


In [ ]:
tissue_assay = pd.merge(assaytissueData, tissueData, on='tissue_id', how='left')
tissue_assay = pd.merge(assayData, tissue_assay, on='assay_id', how='left')
tissue_assay.head()

In [ ]:
pepsql = "SELECT peptide_to_protein.peptide_id, peptide_to_protein.uniprot_id FROM peptide_to_protein"
pepData = pd.read_sql_query(pepsql, conn)
pepData.head()

In [4]:
seqsql = "SELECT uniprot_id, length FROM protein WHERE length IS NOT NULL"
seqData = pd.read_sql_query(seqsql, conn)
seqData.head()

,uniprot_id,length
0,A0A024RBG1,181.0
1,A0A075B6H7,116.0
2,A0A075B6H8,117.0
3,A0A075B6H9,119.0
4,A0A075B6I0,122.0


In [5]:
seqData['length'] = pd.to_numeric(seqData['length'], errors='coerce')

In [ ]:
proteotypicData = pepData.groupby("peptide_id").filter(lambda x: len(x) == 1)
#proteotypicData

In [ ]:
proteins = proteotypicData.groupby("uniprot_id").filter(lambda x: len(x) > 2)
proteins.shape

In [ ]:
non_human_proteins = ['TRYP_PIG', 'TRY2_BOVIN','TRY1_BOVIN','SSPA_STAAU','SRPP_HEVBR','REF_HEVBR', 'ADH1_YEAST', 'ALBU_BOVIN', 'CAS1_BOVIN', 'CAS2_BOVIN', 'CASK_BOVIN', 'CASB_BOVIN', 'OVAL_CHICK', 'ALDOA_RABIT', 'BGAL_ECOLI', 'CAH2_BOVIN', 'CTRA_BOVIN', 'CTRB_BOVIN', 'CYC_HORSE', 'DHE3_BOVIN', 'GAG_SCVLA', 'GFP_AEQVI', 'K1C15_SHEEP', 'K1M1_SHEEP', 'K2M2_SHEEP', 'K2M3_SHEEP', 'KRA3A_SHEEP', 'KRA3_SHEEP', 'KRA61_SHEEP', 'LALBA_BOVIN', 'LYSC_CHICK', 'LYSC_LYSEN', 'MYG_HORSE', 'K1M2_SHEEP', 'K2M1_SHEEP']

In [ ]:
proteins = proteins[~proteins['uniprot_id'].isin(non_human_proteins)]

In [ ]:
protData = pd.merge(tissue_assay, proteins, on = 'peptide_id').sort_values(['assay_id','uniprot_id'])
protData.head(10)
#protData

In [ ]:
del protData['peptide_id']
del protData['tissue_id']
del protData['cell_type']
del protData['disease_status']

In [ ]:
protData.head(10)

Split per tissue, each protein should be found in 90% of the assays of this tissue

In [3]:
#protData.to_csv('protData_tissue_0608.csv', sep = ';', index=False)
protData = pd.read_csv('protData_tissue_0608.csv', sep = ';')

In [4]:
assays = protData['tissue_name'].unique()
DataFrameDict = {elem : pd.DataFrame for elem in assays}
for key in DataFrameDict.keys():
    DataFrameDict[key] = protData[:][protData['tissue_name'] == key]

In [5]:
DataFrameDict.items()

dict_items([('Brain',           assay_id  quantification tissue_name uniprot_id
0            13988             2.0       Brain     A2RRP1
1            13988             1.0       Brain     A5YM72
2            13988             1.0       Brain     O00483
3            13988             1.0       Brain     O00483
4            13988             1.0       Brain     O14495
...            ...             ...         ...        ...
14542106     28261             1.0       Brain     Q9Y6V0
14542107     28261             1.0       Brain     Q9Y6V0
14542108     28261             1.0       Brain     Q9Y6V0
14542109     28261             1.0       Brain     Q9Y6V0
14542110     28261             1.0       Brain     Q9Y6X5

[734106 rows x 4 columns]), ('Blood',           assay_id  quantification tissue_name uniprot_id
28061        14003             1.0       Blood     A2RRP1
28062        14003             1.0       Blood     A5YKK6
28063        14003             1.0       Blood     A6NGG8
28064      

In [8]:
DataFrameDict2 = DataFrameDict.copy()
reduction = []
for key in DataFrameDict2.keys():
    perc = math.floor(0.90 * len(pd.unique(DataFrameDict2[key]['assay_id'])))
    before= DataFrameDict2[key]['uniprot_id'].nunique()
    DataFrameDict2[key] = DataFrameDict2[key].groupby('uniprot_id').filter(lambda x : len(x)>perc)
    after= DataFrameDict2[key]['uniprot_id'].nunique()
    reduction.append(before-after)
    print('{} is done'.format(key))


In [ ]:
print(reduction)

In [ ]:
sum(reduction)

In [ ]:
filteredData = pd.DataFrame()
for key in DataFrameDict2.keys():
    filteredData = filteredData.append(DataFrameDict2[key])

In [ ]:
del filteredData['tissue_name']

In [ ]:
filteredData.head()

In [7]:
#filteredData.to_csv('filtered_tissue_0608.csv', index=False, sep=',')
filteredData = pd.read_csv('filtered_tissue_0608.csv',sep=',')
filteredData.head()

Split data per assay

In [ ]:
assays = filteredData['assay_id'].unique()
DataFrameDict3 = {elem : pd.DataFrame for elem in assays}
for key in DataFrameDict3.keys():
    DataFrameDict3[key] = filteredData[:][filteredData['assay_id'] == key]

In [ ]:
DataFrameDict4 = DataFrameDict3.copy()

In [ ]:
for key in DataFrameDict4.keys():
    sumSaf = 0
    assay = DataFrameDict4[key]
    assay.pop('assay_id')
    # calculate sum of spectral counts for each protein
    grouped = DataFrameDict4[key].groupby('uniprot_id').sum().reset_index()
    seqAddedDF = pd.merge(grouped, seqData, on = 'uniprot_id')
    seqAddedDF.insert(loc = 2, column = 'SAF', value = 0)
    seqAddedDF.insert(loc = 3, column = 'NSAF', value = 0)

    # calculate SAF score for each protein by dividing sum of spectral counts by protein length
    for index, row in seqAddedDF.iterrows():
        saf = row['quantification']/row['length']
        seqAddedDF.loc[index, 'SAF']  = saf
        # calculate sum of SAF scores in assay
        sumSaf = sumSaf + saf

    # Calculate NSAF score by normalizing each SAF score
    seqAddedDF['NSAF'] = seqAddedDF['SAF']/ sumSaf

    del seqAddedDF['length']
    del seqAddedDF['quantification']
    del seqAddedDF['SAF']
    seqAddedDF.insert(loc = 0, column = 'assay_id', value = key)
    DataFrameDict4[key] = seqAddedDF


In [ ]:
proteinData = pd.DataFrame()
for key in DataFrameDict4.keys():
    proteinData = proteinData.append(DataFrameDict4[key])


In [ ]:
proteinData.head()

In [ ]:
# write NSAF proteome to file
proteinData.to_csv(path_or_buf = '../Atlas_and_parser/Atlas_output/filtered_90_proteome_nsaf_update0804.csv', index = False)